# Mixing Sina Records and ORMS

Sina offers great flexibility and does not enforce any limitation on the numbers of columns you can have on a Record table. But as a result this can make it difficult to use Sina Records with others ORM-based tools.

Fortunately sqlalchemy offers the notion of *virtual columns* and one can use these in conjuntion with Sina's `SQLRecord` class

## Setting up the problem

To illsutrate how to use Sina records as sqlalchemy objects we are going to use a very simple example.

We are going to create `simulation`-typed records in Sina. These records will only contain 2 data `simulation_name` and `status`. Where `simulation_name` is a `string` and `status` a `boolean`.

Let's create a new store and put 20 records in it. The record ids will be 0 through 19.



In [1]:
import sina
from sina.model import Record

# create store an empty store
uri = 'orm.sqlite'
store = sina.connect(uri)
store.delete_all_contents(force="SKIP PROMPT")

# create dummy records
for i in range(20):
    rec = Record(f"{i}", "sina")
    rec.add_data("simulation_name", f"sim_{i}")
    rec.add_data("status", i % 2 == 0)
    rec.add_data("score", i + 0.5)
    store.records.insert(rec)


## Making Sina internal tables look like regular Sqlalchemy classes

### Virtual columns

Now if we where to look at the **orm.sqlite** store in a viewer the records will only show `id`, `type` and `raw` as columns. And our sqlalchemy wouldn't be able to ask anything about `simulation_name` or `status`

In order to bypass we are using sqlalchemy's `virtual columns`.

We need to create a new class containing the two virtual columns, using sqlalchemy's `hybrid_property`. 

In [2]:
from sina.datastores.sql_schema import Record as SQLRecord
from sqlalchemy.ext.hybrid import hybrid_method, hybrid_property
from sqlalchemy import Column
from functools import partial

# Now let's create our columns as hybrid_properties
class Simulation(SQLRecord):
    __sina_records = store.records
    def get_column_from_sina_record(self, column_name):
        rec = self.__sina_records.get(self.id)
        return rec["data"][column_name]["value"]
    
    @hybrid_method
    def like_column_name(self, column_name, value):
        pattern = '%"'+column_name+'":{"value":' + repr(value).replace("'", "\"") + '}%'
        return Column('raw').like(pattern)

    def __init__(self, id, type, raw=None):
        super().__init__(id, type, raw)

for col_name in ["simulation_name", "status"]:
    l = partial(Simulation.get_column_from_sina_record, column_name=col_name)
    h = hybrid_property(l)
    h.__name__ = col_name
    setattr(Simulation, col_name, h)



### Query methods

Notice the addtion of the `like_column_name` which allows us the query on a column name. 

We can now query our Sina store using the regular sqlalchemy queries and the `like_column_name` function:

For example let's ask for all simulation starting with `sim_1` or all the simulations that have a `True` status

In [3]:
from sqlalchemy import create_engine, select, and_
from sqlalchemy.orm import sessionmaker
engine = create_engine(f"sqlite:///{uri}")
Session = sessionmaker(bind=engine)
session = Session()

results = session.query(Simulation).filter(Simulation.like_column_name("simulation_name", "sim_3")).all()
print("All:",results)

for obj in results:
    print(obj.id, obj.simulation_name, obj.status)

results = session.query(Simulation).filter(Simulation.like_column_name("simulation_name", 'sim_1%')).all()
print("All:",results)

for obj in results:
    print(obj.id, obj.simulation_name, obj.status)
results = session.query(Simulation).filter(Simulation.like_column_name("status", True)).all()
print("All:",results)

for obj in results:
    print(obj.id, obj.simulation_name, obj.status)

All: [SQL Schema Record <id=3, type=sina>]
3 sim_3 False
All: [SQL Schema Record <id=1, type=sina>, SQL Schema Record <id=10, type=sina>, SQL Schema Record <id=11, type=sina>, SQL Schema Record <id=12, type=sina>, SQL Schema Record <id=13, type=sina>, SQL Schema Record <id=14, type=sina>, SQL Schema Record <id=15, type=sina>, SQL Schema Record <id=16, type=sina>, SQL Schema Record <id=17, type=sina>, SQL Schema Record <id=18, type=sina>, SQL Schema Record <id=19, type=sina>]
1 sim_1 False
10 sim_10 True
11 sim_11 False
12 sim_12 True
13 sim_13 False
14 sim_14 True
15 sim_15 False
16 sim_16 True
17 sim_17 False
18 sim_18 True
19 sim_19 False
All: [SQL Schema Record <id=0, type=sina>, SQL Schema Record <id=2, type=sina>, SQL Schema Record <id=4, type=sina>, SQL Schema Record <id=6, type=sina>, SQL Schema Record <id=8, type=sina>, SQL Schema Record <id=10, type=sina>, SQL Schema Record <id=12, type=sina>, SQL Schema Record <id=14, type=sina>, SQL Schema Record <id=16, type=sina>, SQL Sche

### Sina's implementation

This could become tedious for Record with lots of data. Fortunately Sina provides a utility to automatically detect the possible column names: `sina.utils.make_sina_sqlalchemy_class` 

You can pass it a list of the column names to virtualize or simpy a Sina Record.

In addition the class provides `greater_column_name`, `greater_equal_column_name`, `less_column_name`, `less_equal_column_name` that allow to filter on values of virtual column and require join.

The following calls would have create the same class as above (plus the 4 functions above):

In [4]:
from sina.datastores.sql_schema import ScalarData
Simulation = sina.utils.make_sina_sqlalchemy_class(store, ["simulation_name", "status", "score"])
rec = store.records.get("1")
Simulation = sina.utils.make_sina_sqlalchemy_class(store, rec)

results = session.query(Simulation).join(ScalarData).filter(Simulation.greater_column_name("score", 5.))
print("Results:", results)
for obj in results:
    print(obj.id, obj.score)


Results: SELECT "Record".id AS "Record_id", "Record".type AS "Record_type", "Record".raw AS "Record_raw" 
FROM "Record" JOIN "ScalarData" ON "Record".id = "ScalarData".id 
WHERE "ScalarData".name = ? AND "ScalarData".value > ?
5 5.5
6 6.5
7 7.5
8 8.5
9 9.5
10 10.5
11 11.5
12 12.5
13 13.5
14 14.5
15 15.5
16 16.5
17 17.5
18 18.5
19 19.5


/usr/WS2/cdoutrix/mambaforge/envs/kosh/lib/python3.11/site-packages/sina/utils.py:1239: SAWarning: This declarative base already contains a class with the same class name and module name as sina.utils.SinaSQLAlchemy, and will be replaced in the string-lookup table.
  class SinaSQLAlchemy(SQLRecord):


### Adding your own methods

As mentioned above you might need to create special query function. In that instance you can use inheritance

In [5]:
class SuperSim(Simulation):
    def get_record(self):
        return self.__sina_records.get(self.id)
    @hybrid_method
    def inbetween_column_name(self, column_name, value1, value2):
        return and_(and_(ScalarData.name == column_name, ScalarData.value > value1), ScalarData.value < value2)
results = session.query(SuperSim).join(ScalarData).filter(SuperSim.inbetween_column_name("score", 5., 8.))
print("Results:", results)
for obj in results:
    print(obj.id, obj.score)


Results: SELECT "Record".id AS "Record_id", "Record".type AS "Record_type", "Record".raw AS "Record_raw" 
FROM "Record" JOIN "ScalarData" ON "Record".id = "ScalarData".id 
WHERE "ScalarData".name = ? AND "ScalarData".value > ? AND "ScalarData".value < ?
5 5.5
6 6.5
7 7.5


## Bringing it all together: Mixing with non-sina tables

At this point we now how to create classes on top of Sina Record class. It's now time to mix them with *regular* sqlaalchemy tables.

### Introduction and IMPORTANT WARNING

Sqlalchemy has a `Base` class to use for all custom implementations. Unfortunately

```python
from sqlalchemy.ext.declarative import declarative_base  # pylint: disable=import-error
Base = declarative_base()
```

Can only be called once has the `Base` object will then collect informtion from classes created based on itself.

So we ***MUST*** use Sina's `Base` class for all of our declarations

In this example we are going to create a `Model` class that can be linked to some Sina `Simulation` created above. We will use the same store.

### Creating the Model

We are creating a Model class linked a simulation (Sina Record), the model also contains a `quality` column


In [6]:
from sina.datastores.sql_schema import Base # IMPORTANT TO USE THIS Base since the Record table is already in there
from sqlalchemy.orm import relationship
from sqlalchemy import Float, Integer, ForeignKey, String

class Model(Base):
    __tablename__ = "__models__"
    id = Column(Integer, primary_key=True) # Regular primary key
    simulation_id = Column(String, ForeignKey("Record.id"))  # Sina's Record primary key is a String
    quality = Column(Float)

    # Define the relationship with Sina table
    simulation = relationship("Record")



Let's create the new table

In [7]:
Base.metadata.create_all(engine)

Ok now let's create a few models that will be linked to different Simulations

In [8]:
model1 = Model(simulation_id="9", quality=13.2) # sim score less than 10
model2 = Model(simulation_id="11", quality=4.8) # sim score greater than 10 but value less than 5
model3 = Model(simulation_id="14", quality=61.9) # simm core greater 10 and value greater 5
model4 = Model(simulation_id="4", quality=4.1)  # Add a model with score > 10 and value < 5

session.add_all([model1, model2, model3, model4])

### Important notes on linking to Sina records

One could think that we could have created the simulations *links* and use this as follow:

```python
sim1 = Simulation(id="9",type="sina")
model1 = Model(simulation=sim1, value=13.2) # sim score less than 10
```

While this code will work, we have noticed that at query time sqlalchemy will try to insert the `sim1` record into the database raising an error since they already exists.

### Querying

Ok now let's say we are interested in Models whose `quality` value is less than 5. but only if the `simulation` `score` is greater than 10.

First let's get all the `Simulation`s of interest

In [9]:

query = session.query(Simulation).join(Model).filter(Model.quality < 5).join(ScalarData).filter(Simulation.greater_column_name("score", 10.))
results = query.all()

for obj in results:
    print(obj.id, obj.score)


11 11.5


Now lt's get all the `Model`s that satisfy these condition

In [10]:

query = session.query(Model).filter(Model.quality < 5).join(Simulation).join(ScalarData).filter(Simulation.greater_column_name("score", 10.))

results = query.all()

for obj in results:
    print(obj.id, obj.quality)


2 4.8
